In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import concatenate, Input, Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import preprocessing
import matplotlib.pyplot as plt
from keras import backend as K
from keras import optimizers
from keras.models import load_model

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def readTrain(filename = 'train_data.csv'):
    df = pd.read_csv(filename)
    train_data = df.iloc[:,1:]
    return train_data
Train = readTrain()
Train

,Date,Time,MidPrice,LastPrice,Volume,BidPrice1,BidVolume1,AskPrice1,AskVolume1
0,2018-06-01,09:30:01,3.7865,3.786,1081100.0,3.786,21500.0,3.787,12900.0
1,2018-06-01,09:30:04,3.7835,3.784,1425900.0,3.783,44200.0,3.784,119200.0
2,2018-06-01,09:30:07,3.7835,3.784,1485500.0,3.783,42700.0,3.784,61100.0
3,2018-06-01,09:30:10,3.7845,3.785,1551600.0,3.784,145900.0,3.785,16900.0
4,2018-06-01,09:30:13,3.7835,3.784,2063300.0,3.783,347700.0,3.784,200100.0
5,2018-06-01,09:30:16,3.7835,3.784,2408600.0,3.783,352600.0,3.784,70700.0
6,2018-06-01,09:30:19,3.7860,3.785,2494800.0,3.784,415000.0,3.788,5800.0
7,2018-06-01,09:30:22,3.7860,3.785,2494800.0,3.784,133200.0,3.788,85900.0
8,2018-06-01,09:30:25,3.7860,3.784,2611500.0,3.784,16500.0,3.788,85900.0
9,2018-06-01,09:30:28,3.7855,3.784,2619200.0,3.784,8800.0,3.787,80100.0


In [3]:
def delete_same(_Train):
    delete_indexes = []
    for i in range(1,len(_Train)):
        if(_Train['Time'][i]==_Train['Time'][i-1]):
            delete_indexes.append(i)
    return _Train.drop(delete_indexes)
Train = delete_same(Train)
Train

,Date,Time,MidPrice,LastPrice,Volume,BidPrice1,BidVolume1,AskPrice1,AskVolume1
0,2018-06-01,09:30:01,3.7865,3.786,1081100.0,3.786,21500.0,3.787,12900.0
1,2018-06-01,09:30:04,3.7835,3.784,1425900.0,3.783,44200.0,3.784,119200.0
2,2018-06-01,09:30:07,3.7835,3.784,1485500.0,3.783,42700.0,3.784,61100.0
3,2018-06-01,09:30:10,3.7845,3.785,1551600.0,3.784,145900.0,3.785,16900.0
4,2018-06-01,09:30:13,3.7835,3.784,2063300.0,3.783,347700.0,3.784,200100.0
5,2018-06-01,09:30:16,3.7835,3.784,2408600.0,3.783,352600.0,3.784,70700.0
6,2018-06-01,09:30:19,3.7860,3.785,2494800.0,3.784,415000.0,3.788,5800.0
7,2018-06-01,09:30:22,3.7860,3.785,2494800.0,3.784,133200.0,3.788,85900.0
8,2018-06-01,09:30:25,3.7860,3.784,2611500.0,3.784,16500.0,3.788,85900.0
9,2018-06-01,09:30:28,3.7855,3.784,2619200.0,3.784,8800.0,3.787,80100.0


In [4]:
def get_diff_bid_ask_volume(_Train):
    diff = _Train['BidVolume1']-_Train['AskVolume1']
    return np.array(diff)
Train['diff_bid_ask_volume'] = get_diff_bid_ask_volume(Train)
Train

,Date,Time,MidPrice,LastPrice,Volume,BidPrice1,BidVolume1,AskPrice1,AskVolume1,diff_bid_ask_volume
0,2018-06-01,09:30:01,3.7865,3.786,1081100.0,3.786,21500.0,3.787,12900.0,8600.0
1,2018-06-01,09:30:04,3.7835,3.784,1425900.0,3.783,44200.0,3.784,119200.0,-75000.0
2,2018-06-01,09:30:07,3.7835,3.784,1485500.0,3.783,42700.0,3.784,61100.0,-18400.0
3,2018-06-01,09:30:10,3.7845,3.785,1551600.0,3.784,145900.0,3.785,16900.0,129000.0
4,2018-06-01,09:30:13,3.7835,3.784,2063300.0,3.783,347700.0,3.784,200100.0,147600.0
5,2018-06-01,09:30:16,3.7835,3.784,2408600.0,3.783,352600.0,3.784,70700.0,281900.0
6,2018-06-01,09:30:19,3.7860,3.785,2494800.0,3.784,415000.0,3.788,5800.0,409200.0
7,2018-06-01,09:30:22,3.7860,3.785,2494800.0,3.784,133200.0,3.788,85900.0,47300.0
8,2018-06-01,09:30:25,3.7860,3.784,2611500.0,3.784,16500.0,3.788,85900.0,-69400.0
9,2018-06-01,09:30:28,3.7855,3.784,2619200.0,3.784,8800.0,3.787,80100.0,-71300.0


In [5]:
def get_diff_volume_price(_Train):
    _Train = _Train.values
    diff_volume = [0]
    diff_mid_price = [0]
    diff_ask_price = [0]
    diff_bid_price = [0]
    for i in range(1,len(_Train)):
        diff_volume.append(_Train[i,4] - _Train[i-1,4])
        diff_mid_price.append(_Train[i,2] - _Train[i-1,2])
        diff_ask_price.append(_Train[i,7] - _Train[i-1,7])
        diff_bid_price.append(_Train[i,5] - _Train[i-1,5])
    for i in range(len(diff_volume)):
        if(diff_volume[i]<0):
            diff_volume[i] = 0
    return np.array(diff_volume),np.array(diff_mid_price),np.array(diff_ask_price), np.array(diff_bid_price)
Train['diff_volume'], Train['diff_mid_price'],Train['diff_ask_price'],Train['diff_bid_price'] = get_diff_volume_price(Train)
Train

,Date,Time,MidPrice,LastPrice,Volume,BidPrice1,BidVolume1,AskPrice1,AskVolume1,diff_bid_ask_volume,diff_volume,diff_mid_price,diff_ask_price,diff_bid_price
0,2018-06-01,09:30:01,3.7865,3.786,1081100.0,3.786,21500.0,3.787,12900.0,8600.0,0.0,0.0000,0.000,0.000
1,2018-06-01,09:30:04,3.7835,3.784,1425900.0,3.783,44200.0,3.784,119200.0,-75000.0,344800.0,-0.0030,-0.003,-0.003
2,2018-06-01,09:30:07,3.7835,3.784,1485500.0,3.783,42700.0,3.784,61100.0,-18400.0,59600.0,0.0000,0.000,0.000
3,2018-06-01,09:30:10,3.7845,3.785,1551600.0,3.784,145900.0,3.785,16900.0,129000.0,66100.0,0.0010,0.001,0.001
4,2018-06-01,09:30:13,3.7835,3.784,2063300.0,3.783,347700.0,3.784,200100.0,147600.0,511700.0,-0.0010,-0.001,-0.001
5,2018-06-01,09:30:16,3.7835,3.784,2408600.0,3.783,352600.0,3.784,70700.0,281900.0,345300.0,0.0000,0.000,0.000
6,2018-06-01,09:30:19,3.7860,3.785,2494800.0,3.784,415000.0,3.788,5800.0,409200.0,86200.0,0.0025,0.004,0.001
7,2018-06-01,09:30:22,3.7860,3.785,2494800.0,3.784,133200.0,3.788,85900.0,47300.0,0.0,0.0000,0.000,0.000
8,2018-06-01,09:30:25,3.7860,3.784,2611500.0,3.784,16500.0,3.788,85900.0,-69400.0,116700.0,0.0000,0.000,0.000
9,2018-06-01,09:30:28,3.7855,3.784,2619200.0,3.784,8800.0,3.787,80100.0,-71300.0,7700.0,-0.0005,-0.001,0.000


In [6]:
def get_label(_Train):
    _Train = _Train.values
    label = []
    for i in range(len(_Train) - 20 -10):
        label.append(_Train[i+10:i+30,2].mean()-_Train[i+9,2])
    return np.array(label)
Label = get_label(Train)
Label

array([-0.00055 , -0.001075, -0.0022  , ...,  0.000475,  0.000525,
        0.000625])

In [7]:
# Train = Train.drop(columns=['Volume','BidPrice1','AskPrice1'])
Train['diff_mid_price'] = Train['diff_mid_price']/ Train['MidPrice']
Train['diff_ask_price'] = Train['diff_ask_price']/ Train['AskPrice1']
Train['diff_bid_price'] = Train['diff_bid_price']/ Train['BidPrice1']
Train

,Date,Time,MidPrice,LastPrice,Volume,BidPrice1,BidVolume1,AskPrice1,AskVolume1,diff_bid_ask_volume,diff_volume,diff_mid_price,diff_ask_price,diff_bid_price
0,2018-06-01,09:30:01,3.7865,3.786,1081100.0,3.786,21500.0,3.787,12900.0,8600.0,0.0,0.000000,0.000000,0.000000
1,2018-06-01,09:30:04,3.7835,3.784,1425900.0,3.783,44200.0,3.784,119200.0,-75000.0,344800.0,-0.000793,-0.000793,-0.000793
2,2018-06-01,09:30:07,3.7835,3.784,1485500.0,3.783,42700.0,3.784,61100.0,-18400.0,59600.0,0.000000,0.000000,0.000000
3,2018-06-01,09:30:10,3.7845,3.785,1551600.0,3.784,145900.0,3.785,16900.0,129000.0,66100.0,0.000264,0.000264,0.000264
4,2018-06-01,09:30:13,3.7835,3.784,2063300.0,3.783,347700.0,3.784,200100.0,147600.0,511700.0,-0.000264,-0.000264,-0.000264
5,2018-06-01,09:30:16,3.7835,3.784,2408600.0,3.783,352600.0,3.784,70700.0,281900.0,345300.0,0.000000,0.000000,0.000000
6,2018-06-01,09:30:19,3.7860,3.785,2494800.0,3.784,415000.0,3.788,5800.0,409200.0,86200.0,0.000660,0.001056,0.000264
7,2018-06-01,09:30:22,3.7860,3.785,2494800.0,3.784,133200.0,3.788,85900.0,47300.0,0.0,0.000000,0.000000,0.000000
8,2018-06-01,09:30:25,3.7860,3.784,2611500.0,3.784,16500.0,3.788,85900.0,-69400.0,116700.0,0.000000,0.000000,0.000000
9,2018-06-01,09:30:28,3.7855,3.784,2619200.0,3.784,8800.0,3.787,80100.0,-71300.0,7700.0,-0.000132,-0.000264,0.000000


In [8]:
def get_train_and_label(train, label):
    train = train.values
    _train = []
    _label = []
    for i in range(len(train)-30):
        if(train[i+30,0]!=train[i,0] or ('13:' in train[i+30,1][:3]) and '11:' in train[i,1][:3]):
            i = i+30
        else:
            _train.append(train[i,:])
            _label.append(label[i])
    for i in range(len(train)-30,len(train)):
        _train.append(train[i,:])
    return np.array(_train),np.array(_label)
train_origin, label_origin = get_train_and_label(Train, Label)
print(train_origin.shape)
print(label_origin.shape)

(421987, 14)
(421957,)


In [9]:
train_origin = train_origin[:,2:]
train_origin

array([[3.7865, 3.786, 1081100.0, ..., 0.0, 0.0, 0.0],
       [3.7835, 3.784, 1425900.0, ..., -0.0007929166116030431,
        -0.0007928118393234974, -0.0007930214115781426],
       [3.7835, 3.784, 1485500.0, ..., 0.0, 0.0, 0.0],
       ...,
       [3.5085, 3.509, 301120515.0, ..., 0.0, 0.0, 0.0],
       [3.5095, 3.509, 302052596.0, ..., 0.00028494087476845414,
        0.00028490028490025355, 0.00028498147620401536],
       [3.5085, 3.509, 302266815.0, ..., -0.0002850220892118825,
        -0.00028498147620401536, -0.00028506271379700397]], dtype=object)

In [10]:
# X_scaler = preprocessing.StandardScaler().fit(train_origin)
# train_data = X_scaler.transform(train_origin)
train_data = train_origin

In [11]:
label_origin = np.reshape(label_origin,(-1,1))
Y_scaler = preprocessing.MinMaxScaler()
y_train_data = Y_scaler.fit_transform(label_origin)
# y_train_data = label_origin

In [12]:
def buildTrain(Train_data, pastDay=10, futureDay=20):
	x_train = []
	for i in range(Train_data.shape[0]-futureDay-pastDay):
		x_train.append(Train_data[i:i+pastDay,:])
	return np.array(x_train)
X_train = buildTrain(train_data)
print(X_train.shape)

(421957, 10, 12)


In [13]:
for i in range(len(X_train)):
    X_train[i,:,:] = preprocessing.StandardScaler().fit_transform(X_train[i,:,:])

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [14]:
X_train

array([[[1.3690706925144218, 2.2360679774994585, -1.7060623011443719,
         ..., 0.07595542129424322, 0.00022101286295403424,
         0.18575456782368646],
        [-1.1201487484207566, -0.7453559925000401, -1.0819439373420745,
         ..., -2.198951599447152, -1.7937719153614555,
         -2.599827801970677],
        [-1.1201487484207566, -0.7453559925000401, -0.9740626888426054,
         ..., 0.07595542129424322, 0.00022101286295403424,
         0.18575456782368646],
        ...,
        [0.9542007856917694, 0.7453559925000401, 0.8528591922062891,
         ..., 0.07595542129424322, 0.00022101286295403424,
         0.18575456782368646],
        [0.9542007856917694, -0.7453559925000401, 1.0640964690500483,
         ..., 0.07595542129424322, 0.00022101286295403424,
         0.18575456782368646],
        [0.5393308788691167, -0.7453559925000401, 1.0780341471279997,
         ..., -0.302995431212146, -0.5973029058416648,
         0.18575456782368646]],

       [[-1.1358152736598388, -

In [16]:
def shuffle(X,Y):
	randomList = np.arange(X.shape[0])
	#np.random.shuffle(randomList)
	return X[randomList], Y[randomList]

# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X_train, y_train_data)

In [17]:
def splitData(X,Y,rate):
	X_train = X[int(X.shape[0]*rate):]
	Y_train = Y[int(Y.shape[0]*rate):]
	X_val = X[:int(X.shape[0]*rate)]
	Y_val = Y[:int(Y.shape[0]*rate)]
	return X_train, Y_train, X_val, Y_val
# split training data and validation data
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

In [18]:
def buildManyToOneModel(shape1, shape2):
    input1 = Input(shape=(shape1[1], shape1[2]))
    x1 = LSTM(32, activation = 'tanh', return_sequences=True)(input1)
    input2 = Input(shape=(shape2[1], shape2[2]))
    x2 = LSTM(32, activation = 'tanh', return_sequences=True)(input2)
    
    x = concatenate([x1,x2],axis = -1)
    x = LSTM(32, activation = 'tanh')(x)
    x = Dense(32, activation = 'tanh')(x)
    output = Dense(1, activation = 'tanh')(x)
    model = Model(inputs = [input1, input2], outputs = output)
    model.compile(optimizer='adam',
              loss='mse')
    model.summary()
    return model

In [22]:
model = buildManyToOneModel(X_train[:,:,:7].shape, X_train[:,:,7:].shape)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 10, 7)        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 10, 5)        0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 10, 32)       5120        input_3[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LSTM)                   (None, 10, 32)       4864        input_4[0][0]                    
__________________________________________________________________________________________________
concatenat

In [23]:
callback = EarlyStopping(monitor='loss', patience=3, verbose=1, mode="auto")
model.fit([X_train[:,:,:7],X_train[:,:,7:]], Y_train, epochs=5, batch_size=64, validation_data=([X_val[:,:,:7],X_val[:,:,7:]], Y_val), callbacks=[callback])

Train on 379762 samples, validate on 42195 samples
Epoch 1/5
379762/379762 [==============================] - 371s 977us/step - loss: 0.0038 - val_loss: 0.0021
Epoch 2/5
379762/379762 [==============================] - 374s 984us/step - loss: 0.0035 - val_loss: 0.0020
Epoch 3/5
379762/379762 [==============================] - 390s 1ms/step - loss: 0.0035 - val_loss: 0.0020
Epoch 4/5
379762/379762 [==============================] - 369s 972us/step - loss: 0.0034 - val_loss: 0.0020
Epoch 5/5
379762/379762 [==============================] - 366s 965us/step - loss: 0.0034 - val_loss: 0.0019


In [23]:
model.save('multi_LSTM_preprocess.h5')

In [34]:
test_X_pd = readTrain('test_data.csv').iloc[1420:,:]
_test_X_pd = test_X_pd.values
test_X_pd['diff_bid_ask_volume'] = get_diff_bid_ask_volume(test_X_pd)
test_X_pd['diff_volume'], test_X_pd['diff_mid_price'],test_X_pd['diff_ask_price'],test_X_pd['diff_bid_price'] = get_diff_volume_price(test_X_pd)
test_X_pd

,Date,Time,MidPrice,LastPrice,Volume,BidPrice1,BidVolume1,AskPrice1,AskVolume1,diff_bid_ask_volume,diff_volume,diff_mid_price,diff_ask_price,diff_bid_price
1420,2018-10-09,09:30:43,3.3595,3.359,4764000.0,3.359,574500.0,3.360,62900.0,511600.0,0.0,0.0000,0.000,0.000
1421,2018-10-09,09:30:46,3.3595,3.359,4802200.0,3.359,543300.0,3.360,63900.0,479400.0,38200.0,0.0000,0.000,0.000
1422,2018-10-09,09:30:49,3.3595,3.359,4839900.0,3.359,610600.0,3.360,68700.0,541900.0,37700.0,0.0000,0.000,0.000
1423,2018-10-09,09:30:52,3.3595,3.360,5037000.0,3.359,611100.0,3.360,471100.0,140000.0,197100.0,0.0000,0.000,0.000
1424,2018-10-09,09:30:55,3.3595,3.360,5434800.0,3.359,280000.0,3.360,424800.0,-144800.0,397800.0,0.0000,0.000,0.000
1425,2018-10-09,09:30:58,3.3585,3.359,5738100.0,3.358,3800.0,3.359,360800.0,-357000.0,303300.0,-0.0010,-0.001,-0.001
1426,2018-10-09,09:31:01,3.3585,3.359,5758500.0,3.358,3800.0,3.359,56800.0,-53000.0,20400.0,0.0000,0.000,0.000
1427,2018-10-09,09:31:04,3.3570,3.358,5762600.0,3.356,1012300.0,3.358,86300.0,926000.0,4100.0,-0.0015,-0.001,-0.002
1428,2018-10-09,09:31:07,3.3570,3.358,5778800.0,3.356,105400.0,3.358,86000.0,19400.0,16200.0,0.0000,0.000,0.000
1429,2018-10-09,09:31:10,3.3555,3.356,5904000.0,3.355,668900.0,3.356,196400.0,472500.0,125200.0,-0.0015,-0.002,-0.001


In [35]:
test_X_pd['diff_mid_price'] /= test_X_pd['MidPrice']
test_X_pd['diff_ask_price'] /= test_X_pd['AskPrice1']
test_X_pd['diff_bid_price'] /= test_X_pd['BidPrice1']
test_X_pd

,Date,Time,MidPrice,LastPrice,Volume,BidPrice1,BidVolume1,AskPrice1,AskVolume1,diff_bid_ask_volume,diff_volume,diff_mid_price,diff_ask_price,diff_bid_price
1420,2018-10-09,09:30:43,3.3595,3.359,4764000.0,3.359,574500.0,3.360,62900.0,511600.0,0.0,0.000000,0.000000,0.000000
1421,2018-10-09,09:30:46,3.3595,3.359,4802200.0,3.359,543300.0,3.360,63900.0,479400.0,38200.0,0.000000,0.000000,0.000000
1422,2018-10-09,09:30:49,3.3595,3.359,4839900.0,3.359,610600.0,3.360,68700.0,541900.0,37700.0,0.000000,0.000000,0.000000
1423,2018-10-09,09:30:52,3.3595,3.360,5037000.0,3.359,611100.0,3.360,471100.0,140000.0,197100.0,0.000000,0.000000,0.000000
1424,2018-10-09,09:30:55,3.3595,3.360,5434800.0,3.359,280000.0,3.360,424800.0,-144800.0,397800.0,0.000000,0.000000,0.000000
1425,2018-10-09,09:30:58,3.3585,3.359,5738100.0,3.358,3800.0,3.359,360800.0,-357000.0,303300.0,-0.000298,-0.000298,-0.000298
1426,2018-10-09,09:31:01,3.3585,3.359,5758500.0,3.358,3800.0,3.359,56800.0,-53000.0,20400.0,0.000000,0.000000,0.000000
1427,2018-10-09,09:31:04,3.3570,3.358,5762600.0,3.356,1012300.0,3.358,86300.0,926000.0,4100.0,-0.000447,-0.000298,-0.000596
1428,2018-10-09,09:31:07,3.3570,3.358,5778800.0,3.356,105400.0,3.358,86000.0,19400.0,16200.0,0.000000,0.000000,0.000000
1429,2018-10-09,09:31:10,3.3555,3.356,5904000.0,3.355,668900.0,3.356,196400.0,472500.0,125200.0,-0.000447,-0.000596,-0.000298


In [36]:
def set_first_zero(pd):
    for i in range(len(pd)):
        if(i%10==0):
            pd.iloc[i,-1] = pd.iloc[i+1:i+10,-1].mean()
            pd.iloc[i,-2] = pd.iloc[i+1:i+10,-2].mean()
            pd.iloc[i,-3] = pd.iloc[i+1:i+10,-3].mean()
            pd.iloc[i,-4] = pd.iloc[i+1:i+10,-4].mean()
    return pd
test_X_pd = set_first_zero(test_X_pd)
test_X_pd

,Date,Time,MidPrice,LastPrice,Volume,BidPrice1,BidVolume1,AskPrice1,AskVolume1,diff_bid_ask_volume,diff_volume,diff_mid_price,diff_ask_price,diff_bid_price
1420,2018-10-09,09:30:43,3.3595,3.359,4764000.0,3.359,574500.0,3.360,62900.0,511600.0,1.266667e+05,-0.000132,-0.000132,-0.000132
1421,2018-10-09,09:30:46,3.3595,3.359,4802200.0,3.359,543300.0,3.360,63900.0,479400.0,3.820000e+04,0.000000,0.000000,0.000000
1422,2018-10-09,09:30:49,3.3595,3.359,4839900.0,3.359,610600.0,3.360,68700.0,541900.0,3.770000e+04,0.000000,0.000000,0.000000
1423,2018-10-09,09:30:52,3.3595,3.360,5037000.0,3.359,611100.0,3.360,471100.0,140000.0,1.971000e+05,0.000000,0.000000,0.000000
1424,2018-10-09,09:30:55,3.3595,3.360,5434800.0,3.359,280000.0,3.360,424800.0,-144800.0,3.978000e+05,0.000000,0.000000,0.000000
1425,2018-10-09,09:30:58,3.3585,3.359,5738100.0,3.358,3800.0,3.359,360800.0,-357000.0,3.033000e+05,-0.000298,-0.000298,-0.000298
1426,2018-10-09,09:31:01,3.3585,3.359,5758500.0,3.358,3800.0,3.359,56800.0,-53000.0,2.040000e+04,0.000000,0.000000,0.000000
1427,2018-10-09,09:31:04,3.3570,3.358,5762600.0,3.356,1012300.0,3.358,86300.0,926000.0,4.100000e+03,-0.000447,-0.000298,-0.000596
1428,2018-10-09,09:31:07,3.3570,3.358,5778800.0,3.356,105400.0,3.358,86000.0,19400.0,1.620000e+04,0.000000,0.000000,0.000000
1429,2018-10-09,09:31:10,3.3555,3.356,5904000.0,3.355,668900.0,3.356,196400.0,472500.0,1.252000e+05,-0.000447,-0.000596,-0.000298


In [37]:
def get_test_midprice(pd):
    pd = pd.values
    res = []
    for i in range(len(pd)):
        if(i%10==9):
            res.append(pd[i,2])
    return np.array(res)
test_mid_price = get_test_midprice(test_X_pd)
test_mid_price

array([3.3555, 3.3645, 3.3555, 3.3555, 3.3525, 3.3515, 3.3475, 3.3495,
       3.3595, 3.3595, 3.3595, 3.3615, 3.3595, 3.3595, 3.3685, 3.3645,
       3.3665, 3.3645, 3.3565, 3.3595, 3.3565, 3.3545, 3.3555, 3.3585,
       3.3575, 3.3625, 3.3585, 3.3525, 3.3505, 3.3475, 3.3455, 3.3465,
       3.3455, 3.3425, 3.3455, 3.3485, 3.3485, 3.3505, 3.3455, 3.3405,
       3.3455, 3.3485, 3.3505, 3.3495, 3.3605, 3.3635, 3.3665, 3.3615,
       3.3625, 3.3605, 3.3595, 3.3625, 3.3595, 3.3655, 3.359 , 3.3585,
       3.356 , 3.3545, 3.3525, 3.3535, 3.3535, 3.3535, 3.3515, 3.3545,
       3.3525, 3.3555, 3.3545, 3.3545, 3.3485, 3.3485, 3.3485, 3.352 ,
       3.3505, 3.3485, 3.3475, 3.3465, 3.3465, 3.3455, 3.3425, 3.3445,
       3.3445, 3.3425, 3.3445, 3.349 , 3.3495, 3.3475, 3.3415, 3.3375,
       3.3375, 3.3415, 3.3405, 3.3435, 3.3425, 3.3435, 3.3445, 3.3485,
       3.3585, 3.355 , 3.3515, 3.3505, 3.3515, 3.3665, 3.3685, 3.3755,
       3.3685, 3.3685, 3.3675, 3.3695, 3.3755, 3.3745, 3.3695, 3.3665,
      

In [38]:
test_X_pd = test_X_pd.drop(columns=['Date','Time'])

In [39]:
# test_X = X_scaler.fit_transform(test_X_pd)
test_X = test_X_pd.values
test_X = np.reshape(test_X,(858,10,-1))
test_X.shape

(858, 10, 12)

In [40]:
for i in range(len(test_X)):
    test_X[i,:,:] = preprocessing.StandardScaler().fit_transform(test_X[i,:,:])

In [41]:
prediction = model.predict([test_X[:,:,:7],test_X[:,:,7:]])
# prediction = Y_scaler.inverse_transform(prediction)
prediction = np.reshape(prediction,(-1))

In [42]:
prediction

array([ 2.35191023e-04,  5.04491967e-04, -1.81269163e-04,  2.44396098e-04,
       -2.90069613e-04,  2.24563206e-04,  2.26263743e-04,  5.88466704e-04,
        4.95285902e-04,  2.28289573e-05,  3.77638033e-04,  1.22524318e-04,
       -2.87171424e-05, -7.30345928e-05,  5.21683600e-04,  3.64129024e-04,
        1.23630176e-04, -9.89181281e-05, -1.69447478e-04,  2.51115765e-04,
        2.10136568e-05,  3.62514023e-04,  1.45900063e-04, -2.12055253e-04,
        1.38208794e-04, -2.97447405e-05,  3.45441804e-04,  1.32489775e-04,
       -3.79617559e-05,  1.26695028e-04,  1.40922581e-04,  4.50061343e-04,
        1.72048691e-04,  5.55976992e-04,  1.94108448e-04, -1.84328877e-04,
        2.30638558e-04, -1.60543830e-04, -1.62087556e-04,  9.74880895e-05,
       -4.18495620e-05, -5.47154777e-05,  5.47897478e-04,  3.70529218e-04,
        5.15618187e-04,  3.31882911e-04, -2.59442604e-05,  1.91548170e-04,
        1.38520045e-04,  1.06223466e-04,  8.73526878e-05,  3.08196439e-04,
        6.19669387e-04,  

In [43]:
for i in range(len(prediction)):
    prediction[i] += test_mid_price[i]
prediction

array([3.3557353, 3.3650045, 3.3553188, 3.3557444, 3.35221  , 3.3517246,
       3.3477263, 3.3500884, 3.3599954, 3.3595228, 3.3598776, 3.3616226,
       3.3594713, 3.359427 , 3.3690217, 3.364864 , 3.3666236, 3.364401 ,
       3.3563306, 3.3597512, 3.3565211, 3.3548625, 3.355646 , 3.358288 ,
       3.3576381, 3.3624701, 3.3588455, 3.3526325, 3.350462 , 3.3476267,
       3.345641 , 3.34695  , 3.3456721, 3.343056 , 3.345694 , 3.3483157,
       3.3487306, 3.3503394, 3.3453379, 3.3405974, 3.3454583, 3.3484452,
       3.351048 , 3.3498704, 3.3610156, 3.363832 , 3.3664742, 3.3616915,
       3.3626385, 3.3606062, 3.3595874, 3.3628082, 3.3601196, 3.3655174,
       3.3591533, 3.3585835, 3.3562548, 3.3546987, 3.3524742, 3.3537319,
       3.3537304, 3.3535872, 3.3517802, 3.3544762, 3.3524523, 3.3558605,
       3.354948 , 3.3543503, 3.349103 , 3.34863  , 3.3486726, 3.3523846,
       3.3505068, 3.3486996, 3.3475132, 3.3467302, 3.3472571, 3.3459418,
       3.3424835, 3.3447392, 3.344729 , 3.3430586, 

In [44]:
caseid = list(range(143,1001))
df = pd.DataFrame({'caseid':caseid, 'midprice':prediction})
df.to_csv('multi_LSTM_Dense_after_normal.csv',index = False, sep = ',')